In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as func
import torch.nn.init as torch_init
import torch.optim as optim
import pandas as pd
import torchvision
from torchvision import transforms, utils
from torchvision import models
import torch.nn.functional as F

import matplotlib.pyplot as plt
%matplotlib inline

import time
import copy
import pickle

In [2]:
from data_augmentation import *
from adv_generate import *

In [3]:
# Check if CUDA is supported
use_cuda = torch.cuda.is_available()

# Setup GPU optimization if CUDA is supported
if use_cuda:
    computing_device = torch.device("cuda")
    extras = {"num_workers": 1, "pin_memory": True}
    print("CUDA is supported")
else: # Otherwise, train on the CPU
    computing_device = torch.device("cpu")
    extras = False
    print("CUDA NOT supported")

CUDA is supported


In [4]:
load_transform = transforms.Compose([transforms.Resize((224, 224)),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])
                                    # same normalize as EfficientNet paper
# cifar-10 dataset
#trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
#                                        download=False, transform=load_transform)
#trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
#                                          shuffle=True)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=False, transform=load_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False)

In [5]:
# load model
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b1')

# change fc output class numbers
model._fc.out_features = 10


model.load_state_dict(torch.load('model/efficient_net_b1_da_at_P025.pth'))

Loaded pretrained weights for efficientnet-b1


<All keys matched successfully>

In [6]:
def inplace_data_manipulation(data_batch, target_batch, model, p=0.1, eps=0.03):
    '''
    data_batch: n_batch x 3 x 32 x 32
    '''
    for i in range(data_batch.shape[0]):
        data = data_batch[i:(i+1)]
        target = target_batch[i:(i+1)]
        
        rand_num = np.random.rand()
        
        #if rand_num < p:
        data_batch[i] = inplace_adv(data.detach(), target, model, eps=eps)
        #elif rand_num < 2*p:
        #    data_batch[i] = inplace_transform(data[0])
        
    return data_batch

In [7]:
import gc

In [8]:
_ = model.eval()
val_correct = 0

model = model.to(computing_device)

for data, target in testloader:
    data = data.to(computing_device)
    target = target.to(computing_device)
    
    data = inplace_data_manipulation(data, target, model)
    
    output = model(data)
    _, preds = torch.max(output.data, 1)
    val_correct += (preds == target).sum().item()
    
    del data
    del target
    del output
    gc.collect()
    
    
acc = val_correct/10000

In [9]:
#baseline
acc

0.2912

In [14]:
# p=0.1
acc

0.871

In [9]:
# p=0.25
acc

0.8913